In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [10]:
# 加载CSV数据
data = pd.read_csv("./new_data.csv", header=None)
data

,0,1,2,3,4,5,6
0,date(日期),Q(径流),E(蒸发),P(降水),Year,Month,Day
1,1983/1/1,6.5,0.9,0.3,1983,1,1
2,1983/1/2,6.99,0,0.2,1983,1,2
3,1983/1/3,6.99,2.2,2.6,1983,1,3
4,1983/1/4,7.49,0.1,27.7,1983,1,4
...,...,...,...,...,...,...,...
1822,1987/12/27,4.52,0.1,2.4,1987,12,27
1823,1987/12/28,4.59,0.9,1.1,1987,12,28
1824,1987/12/29,1.38,1,1,1987,12,29
1825,1987/12/30,6.84,1.1,0,1987,12,30


In [11]:
data = data[1:]

In [12]:
# 将数据拆分为特征和标签
X = data.iloc[:, [2, 3, 4, 5, 6]].values
y = data.iloc[:, 1].values

In [13]:
# 将数据集拆分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [14]:
# 定义模型集合
models = []
for i in range(10):
    models.append(BayesianRidge())

In [15]:
# 训练模型集合
for model in models:
    model.fit(X_train, y_train)

In [16]:
# 预测结果
y_preds = np.zeros((len(models), len(X_test)))
for i, model in enumerate(models):
    y_preds[i] = model.predict(X_test)

In [17]:
# 计算BMA权重
w = np.zeros(len(models))
for i in range(len(models)):
    y_pred_i = y_preds[i]
    mse_i = mean_squared_error(y_test, y_pred_i)
    alpha_i = 0.5 * np.log((len(data) - 1) / mse_i)
    w[i] = alpha_i
w = w / np.sum(w)

In [18]:
# 计算BMA预测结果
y_bma = np.zeros(len(X_test))
for i, model in enumerate(models):
    y_bma += w[i] * model.predict(X_test)

In [19]:
np.set_printoptions(suppress = True)
float_y_test = np.array([float(x) for x in y_test])
float_y_test

array([ 33.9 ,  32.8 ,   9.57,   9.08,   3.19,   8.5 ,   4.14,   5.62,
        31.5 ,   5.7 ,   4.58,  58.7 ,  22.7 ,  28.1 ,  10.3 ,  28.6 ,
         2.38,   5.89, 112.  ,  40.3 ,   3.79,   7.73,   4.78,   4.81,
        15.8 ,  14.2 ,   3.89,   3.6 ,   9.68,   6.38,   3.83,  17.8 ,
         8.12,   7.01,   2.43,  46.2 ,   3.38,  92.2 ,  12.7 ,  22.2 ,
         2.47,   9.96,  28.1 ,   7.64,  17.2 ,   7.96,   4.12,   6.58,
         3.24,   6.08,  13.3 ,   2.64,  13.5 ,   3.12,  19.3 ,   0.95,
         1.88,   6.84,  22.9 ,   4.11,   9.86,   6.01,  10.9 ,  16.3 ,
         0.71,   4.09,   9.04,  93.9 ,   8.08,  13.3 ,   5.52,   7.02,
         3.77,   8.92,   3.08,   5.69,  31.6 ,  15.2 ,   3.02,  18.8 ,
         6.23,  16.  ,  27.6 ,   1.66,   3.7 ,   6.71,  15.4 ,  72.9 ,
        70.2 ,  13.1 ,   8.09,  36.5 ,  45.8 ,  15.3 ,   6.83,   5.3 ,
         2.44,   4.51,  22.6 ,   8.26,   3.81,  13.9 ,   2.85,   4.31,
         8.83,  60.  ,   3.34,  19.9 ,  39.7 ,  71.  ,   7.3 ,   8.64,
      

In [21]:
# 计算评估指标
mape = np.mean(np.abs((float_y_test - y_bma) / float_y_test)) * 100
nse = 1 - np.sum((float_y_test - y_bma) ** 2) / np.sum((float_y_test - np.mean(float_y_test)) ** 2)
rmspe = np.sqrt(np.mean(((float_y_test - y_bma) / float_y_test) ** 2))
tolerance = 0.1
accuracy = np.sum(np.abs((float_y_test - y_bma) / float_y_test) <= tolerance) / len(float_y_test)

# 输出结果
print(f"MAPE: {mape:.2f}")
print(f"NSE: {nse:.2f}")
print(f"RMSPE: {rmspe:.4f}")
print(f"Accuracy: {accuracy:.2f}")

MAPE: 214.06
NSE: 0.11
RMSPE: 5.0663
Accuracy: 0.08
